In [1]:
import os
from pathlib import Path

import numpy as np
import pandas as pd
from pandas.api.types import CategoricalDtype
import xarray as xr

from fake_data_for_learning.contingency_tables import calculate_contingency_table

## Data wrangling example(s) with [pandas](https://pandas.pydata.org/pandas-docs/stable/) and [xarray](https://docs.xarray.dev/en/stable/index.html)

In [2]:
# Set (local) data directory
datadir = Path(os.getcwd()) / 'data'
path_or_url = datadir / 'default.csv'
if not path_or_url.exists():
    path_or_url = 'https://raw.githubusercontent.com/munichpavel/risk-ai-workshop/main/notebooks/data/default.csv'

In [3]:
df = pd.read_csv(path_or_url, sep=',')
print(f'Loaded data table of shape {df.shape}')
n_records = df.shape[0]
df.head()

Loaded data table of shape (10000, 4)


,gender,occupation,activity,default
0,1,1,0,0
1,1,1,0,0
2,1,1,0,0
3,0,0,1,1
4,1,1,0,0


### Basic slicing and dicing with pandas

See also https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html.

#### Filtering rows

In [4]:
# Create a boolean mask, and calculate subpopulation ratio
mask_female = df['gender'] == 0
subpopulation_ratio = sum(mask_female) / n_records
print(f'Female subpopulation ratio is {subpopulation_ratio}')

Female subpopulation ratio is 0.4857


In [5]:
# or using more sql-like functionality
grouped = df.groupby('gender')
grouped_size = grouped.size()
subpopulation_ratio = grouped_size[0] / n_records
print(f'Female subpopulation ratio is {subpopulation_ratio}')

Female subpopulation ratio is 0.4857


## Data wrangling with xarray example(s)

In [6]:
default_values = [0, 1]
gender_values = [0, 1]
occupation_values = [0, 1]
activity_values = [0, 1]
df_categories = dict(
    default=CategoricalDtype(categories=default_values, ordered=True),
    gender=CategoricalDtype(categories=gender_values, ordered=True),
    activity=CategoricalDtype(categories=activity_values, ordered=True),
    occupation=CategoricalDtype(categories=occupation_values, ordered=True)
)
for column, c_type in df_categories.items():
    df[column] = df[column].astype(c_type)
    
contingency = calculate_contingency_table(df[df_categories.keys()])
contingency

<xarray.DataArray (default: 2, gender: 2, activity: 2, occupation: 2)>
array([[[[1911.,  919.],
         [  80.,   18.]],

        [[ 654., 2820.],
         [  35.,   64.]]],


       [[[   0.,    0.],
         [1641.,  288.]],

        [[   0.,    0.],
         [ 618.,  952.]]]])
Coordinates:
  * default     (default) int64 0 1
  * gender      (gender) int64 0 1
  * activity    (activity) int64 0 1
  * occupation  (occupation) int64 0 1

In [7]:
subpopulation_ratio = contingency.sel(gender=0).sum().values / n_records
print(f'Female population ratio with xarray: {subpopulation_ratio}')

Female population ratio with xarray: 0.4857


## Exercise: Data wrangling with pandas

Difficulty: (*)

Calculate the subpopulation ratio within the artifical [credit default data](https://github.com/munichpavel/risk-ai-workshop/blob/main/notebooks/data/default.csv) of 

* males among total population
* females of occupation 0 ("education") who default (default = 1) among total population
* males of occupation 0 ("education") who default (default=1) among total population

## Exercise: Data wrangling with xarray

Difficulty: (**)

Calculate the subpopulation ratio within the artifical [credit default data](https://github.com/munichpavel/risk-ai-workshop/blob/main/notebooks/data/default.csv) of 

* males among total population
* female education workers (occupation 0) among education workers
* female health workers (occupation 1) among health workers